In [98]:
import pandas as pd
pd.set_option('display.max_rows', 200) 
proposals = pd.read_parquet("../output_data/snapshot/cleaned_proposals.parquet")
# 145 records
# print(proposals.info())
proposals['choices'] = proposals['choices'].apply(lambda x: ', '.join(map(str, x)))

filtered_proposals = proposals[proposals["choices"] == 'For, Against, Abstain']
# 126 records
# filtered_proposals.info()

voting = pd.read_parquet("../output_data/snapshot/votes.parquet")

filtered_votes = pd.merge(
    filtered_proposals, voting, left_on="id", right_on="proposal_id", how="inner"
)
filtered_proposals.info()
# voting.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, 0 to 144
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   id                   126 non-null    object             
 1   title                126 non-null    object             
 2   body                 126 non-null    object             
 3   choices              126 non-null    object             
 4   start                126 non-null    int64              
 5   end                  126 non-null    int64              
 6   snapshot             126 non-null    object             
 7   state                126 non-null    object             
 8   author               126 non-null    object             
 9   scores               126 non-null    object             
 10  votes                126 non-null    int64              
 11  flagged              126 non-null    bool               
 12  discussion           126 no

In [92]:
import pandas as pd

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

pd.set_option('display.float_format', '{:.2f}'.format)

karma_delegates = pd.read_parquet("../output_data/karma/delegates.parquet")
print(karma_delegates.info)
# voting = pd.read_parquet("output_data/snapshot/votes.parquet")
filtered_votes['voter'] = filtered_votes['voter'].str.lower()

voting_merged_df = pd.merge(
    karma_delegates, filtered_votes, left_on="publicAddress", right_on="voter"
)
voting_merged_df = voting_merged_df[['publicAddress', 'choice', 'vp', 'proposal_id']]

# sample
# df = voting_merged_df[voting_merged_df['proposal_id'] == "0x3be7368a662d1cf12fa4da768d626edbc013be0dc7b994fef2e24d9a54e4033a"]

# print(df.describe())

# all voters
# voting = voting[voting['proposal_id'] == "0x3be7368a662d1cf12fa4da768d626edbc013be0dc7b994fef2e24d9a54e4033a"]
count_by_choice = voting.groupby('choice').size()

# delegates voters
# count_by_choice = df.groupby('choice').size()

print(voting_merged_df)

<bound method DataFrame.info of      githubHandle                               publicAddress   daoName  \
0            None  0xababe91a907fadff921720c43abe4098e7245e99  arbitrum   
1            None  0x17296956b4e07ff8931e4ff4ea06709fab70b879  arbitrum   
2            None  0x563d27dd2e9f36157c658501222a7cb23f690c28  arbitrum   
3            None  0x382cab376440f92d4943f8b47dd45645cf98be0c  arbitrum   
4            None  0xad16ebe6ffc7d96624a380f394cd64395b0c6144  arbitrum   
...           ...                                         ...       ...   
1417         None  0x5aaaef91f93be4de932b8e7324abbf9f26daa706  arbitrum   
1418         None  0x8506970ca968367e87c225c2f63c0a26f5aead24  arbitrum   
1419         None  0x683a4f9915d6216f73d6df50151725036bd26c02  arbitrum   
1420         None  0x8a3e9846df0cdc723c06e4f0c642ffff82b54610  arbitrum   
1421         None  0x00000000000000000000000000000000000a4b86  arbitrum   

                   ensName twitterHandle discourseHandle  isForumVe

## create pivot table

In [24]:
import os

value_mapping = {"1": "For", "2": "Against", "3": "Abstain"}

pivoted_df = voting_merged_df.pivot(index='publicAddress', columns='proposal_id', values='choice')
pivoted_df.columns = [f'proposal_{col}_choice' for col in pivoted_df.columns]
for column in pivoted_df.columns: 
    pivoted_df[column] = pivoted_df[column].map(value_mapping)
    
pivoted_df.reset_index(inplace=True)


print(pivoted_df.head())

pivoted_df.to_csv("pivot_votes.csv", index=False)


                                publicAddress  \
0  0x001c3b137c68f592952b82786096148390e9030d   
1  0x009c32960e474117d85fab65494729395c0959cf   
2  0x00a65468ce2bfc7365269ea2f39cc24ade522610   
3  0x01cf9fd2efa5fdf178bd635c3e2adf25b2052712   
4  0x01f761805c90bc8abfaf504075afdb87e1e5dc08   

  proposal_0x029f9208ec4560c5c4868bcf1a5633ffb15aa69f5000fbaa22146632b440188a_choice  \
0                                                NaN                                   
1                                                NaN                                   
2                                                NaN                                   
3                                                NaN                                   
4                                                For                                   

  proposal_0x03e61c8225487f2ff6305c91ce37af31853e6c09f557ad6c10db188d21e85f24_choice  \
0                                                NaN                                   


/var/folders/gl/3nbwj5r1183_gfp3xql0hclr0000gn/T/ipykernel_86443/1548847374.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_df.reset_index(inplace=True)


In [27]:
from kmodes import kmodes

pivoted_df.fillna("No vote", inplace=True)

# Number of clusters
k = 4

# Selecting features for clustering
X = pivoted_df.iloc[:, 1:].astype(str)
km = kmodes.KModes(n_clusters=4, init='Huang', n_init=5, verbose=0)
pivoted_df['cluster'] = km.fit_predict(X)
# pivoted_df
print(pivoted_df.groupby('cluster').size())

pivoted_df.to_csv("data/clustered_delegates.csv", index=False)

cluster
0    263
1     34
2    441
3     74
dtype: int64


In [39]:
df = pd.read_csv('data/clustered_delegates.csv')

df[["publicAddress", "cluster"]].head()

,publicAddress,cluster
0,0x001c3b137c68f592952b82786096148390e9030d,0
1,0x009c32960e474117d85fab65494729395c0959cf,2
2,0x00a65468ce2bfc7365269ea2f39cc24ade522610,2
3,0x01cf9fd2efa5fdf178bd635c3e2adf25b2052712,2
4,0x01f761805c90bc8abfaf504075afdb87e1e5dc08,3


In [97]:
from sklearn.decomposition import PCA
import plotly.express as px

df = pd.read_csv('data/clustered_delegates.csv')
value_mapping = {"For": 1, "Against": 2, "Abstain": 3, "No vote" : 0}

for column in df.columns[1:-1]: 
    df[column] = df[column].map(value_mapping)
df.fillna(0, inplace=True)
print(df[["publicAddress", "cluster"]].head())

pca = PCA(n_components=2)

feature = pca.fit_transform(df.iloc[:, 1:-1])
df['feature1'] = feature[:, 0]
df['feature2'] = feature[:, 1]
df = df[['publicAddress', 'cluster', 'feature1', 'feature2']]
df.to_csv("data/clustered_pca_delegates.csv", index=False)


fig = px.scatter(df, x=feature[:, 0], y=feature[:, 1], color=df['cluster'], hover_name='publicAddress', title="Clustered Delegates")
fig.show()
# plt.figure(figsize=(10, 6))
# plt.scatter(feature[:, 0], feature[:, 1], c=df['cluster'], cmap='viridis', marker='o', alpha=0.7)

# plt.title('K-means Clustering with Address Labels')
# plt.show()

                                publicAddress  cluster
0  0x001c3b137c68f592952b82786096148390e9030d        0
1  0x009c32960e474117d85fab65494729395c0959cf        2
2  0x00a65468ce2bfc7365269ea2f39cc24ade522610        2
3  0x01cf9fd2efa5fdf178bd635c3e2adf25b2052712        2
4  0x01f761805c90bc8abfaf504075afdb87e1e5dc08        3


/var/folders/gl/3nbwj5r1183_gfp3xql0hclr0000gn/T/ipykernel_86443/1816505939.py:15: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/gl/3nbwj5r1183_gfp3xql0hclr0000gn/T/ipykernel_86443/1816505939.py:16: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [87]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200) 

df['totalForVotes'] = 0
df['totalAgainstVotes'] = 0
df['totalAbstainVotes'] = 0
df['totalNoVotes'] = 0

for column in df.columns[1:-4]: 
    df['totalForVotes'] += df[column].apply(lambda x: 1 if x == 1 else 0)
    df['totalAgainstVotes'] += df[column].apply(lambda x: 1 if x == 2 else 0)
    df['totalAbstainVotes'] += df[column].apply(lambda x: 1 if x == 3 else 0)
    df['totalNoVotes'] += df[column].apply(lambda x: 1 if x == 0 else 0)

dfCleanded = df[['publicAddress', 'cluster', 'totalForVotes', 'totalAgainstVotes', 'totalAbstainVotes', 'totalNoVotes']]


karma_delegates = pd.read_parquet("../output_data/karma/delegates.parquet")
dfCleanded = pd.merge(dfCleanded, karma_delegates, on='publicAddress')
dfCleanded.head()

dfCleanded.to_csv('data/clustered_delegates_total.csv', index=False)

/var/folders/gl/3nbwj5r1183_gfp3xql0hclr0000gn/T/ipykernel_86443/2846446388.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/gl/3nbwj5r1183_gfp3xql0hclr0000gn/T/ipykernel_86443/2846446388.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/gl/3nbwj5r1183_gfp3xql0hclr0000gn/T/ipykernel_86443/2846446388.py:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [88]:
average_totalFor_by_cluster = df.groupby('cluster')[['totalForVotes', 'totalAgainstVotes', 'totalAbstainVotes', 'totalNoVotes']].mean()
print(average_totalFor_by_cluster)

         totalForVotes  totalAgainstVotes  totalAbstainVotes  totalNoVotes
cluster                                                                   
0                 6.56               2.41               0.51        119.87
1                49.50               7.71               8.79         61.97
2                19.27               1.33               0.13        108.22
3                52.01              54.89               9.14         11.64


In [89]:
import plotly.express as px

fig = px.box(df,x='cluster', y="totalForVotes")
fig.show()

fig = px.box(df,x='cluster', y="totalAgainstVotes")
fig.show()

fig = px.box(df,x='cluster', y="totalAbstainVotes")
fig.show()

fig = px.box(df,x='cluster', y="totalNoVotes")
fig.show()

# Cluster 
0 : many no vote
1: less no vote, many For vote, less Against vote
2 : many no vote, a bit For vote than 0
3 : minimum no vote and valid for and agaisnt vote

In [90]:

# karma_delegates[['publicAddress', 'daos']].head(20)

clustered_delegates_total = pd.read_csv('data/clustered_delegates_total.csv')

clustered_delegates_total[clustered_delegates_total['cluster'] == 3].head()

,publicAddress,cluster,totalForVotes,totalAgainstVotes,totalAbstainVotes,totalNoVotes,githubHandle,daoName,ensName,twitterHandle,discourseHandle,isForumVerified,isNominee,score,daos,poap,daoCount,userType,pitch,stats,delegatorCount,delegatedVotes,snapshotDelegatedVotes,voteWeight,firstTokenDelegatedAt,joinDateAt,workstreams,scoreBreakdown,aboutMe,profilePicture,website,realName,status,discordHandle,discordUsername,roles,githubScores,githubScorePercentile,userCreatedAt,acceptedTOS,termsOfAgreement,tracks,discussionThread
4,0x01f761805c90bc8abfaf504075afdb87e1e5dc08,3,83,30,13,2,NaN,arbitrum,thunderbluff.eth,NaN,NaN,False,False,77,hop.eth,0,1,Ind,NaN,"[{'aragonVotesPct': None, 'createdAt': '2023-12-17T14:39:38.233Z', 'deworkPoints': None, 'deworkTasksCompleted': None, 'discordScore': None, 'forumActivityScore': None, 'forumLikesReceived': None, 'forumPostCount': None, 'forumPostsReadCount': None, 'forumTopicCount': None, 'gitcoinHealthScore': None, 'karmaRank': 28, 'karmaScore': 77, 'offChainVotesPct': 99, 'onChainVotesPct': 79, 'percentile': 10.0, 'period': 'lifetime', 'proposalsDiscussed': None, 'proposalsInitiated': None, 'proposalsOnAragon': None, 'proposalsOnSnapshot': 0, 'updatedAt': '2024-03-02T02:26:07.730Z'}]",1,10261,NaN,0.00,2023-03-24T23:15:44.000Z,2023-03-24T23:15:44.000Z,[],"[{'name': 'forumActivityScore', 'pct': 0.0, 'score': 0}\n {'name': 'offChainVotesPct', 'pct': 42.857142857142854, 'score': 33}\n {'name': 'onChainVotesPct', 'pct': 57.14285714285714, 'score': 44}]",NaN,NaN,NaN,NaN,active,NaN,NaN,[],NaN,NaN,2022-06-17T13:50:43.893Z,False,NaN,[],NaN
7,0x03d66c06177276c6cb1c410d8b77c0a628deffc1,3,18,90,1,19,NaN,arbitrum,darkrainds.eth,NaN,NaN,False,False,35,arbitrumfoundation.eth,0,1,Ind,NaN,"[{'aragonVotesPct': None, 'createdAt': '2023-07-21T15:04:06.471Z', 'deworkPoints': None, 'deworkTasksCompleted': None, 'discordScore': None, 'forumActivityScore': None, 'forumLikesReceived': None, 'forumPostCount': None, 'forumPostsReadCount': None, 'forumTopicCount': None, 'gitcoinHealthScore': None, 'karmaRank': 536, 'karmaScore': 35, 'offChainVotesPct': 77, 'onChainVotesPct': 16, 'percentile': None, 'period': 'lifetime', 'proposalsDiscussed': None, 'proposalsInitiated': None, 'proposalsOnAragon': None, 'proposalsOnSnapshot': 0, 'updatedAt': '2024-03-02T02:26:09.070Z'}]",1,0,NaN,NaN,2023-03-24T01:16:26.000Z,2023-03-24T01:16:26.000Z,[],"[{'name': 'forumActivityScore', 'pct': 0.0, 'score': 0}\n {'name': 'offChainVotesPct', 'pct': 74.28571428571429, 'score': 26}\n {'name': 'onChainVotesPct', 'pct': 25.71428571428571, 'score': 9}]",NaN,NaN,NaN,NaN,active,NaN,NaN,[],NaN,NaN,2023-07-21T15:04:05.836Z,False,NaN,[],NaN
11,0x048093338957ead2abeee66b1fe838766658471f,3,49,70,2,7,NaN,arbitrum,dexico.eth,NaN,NaN,False,False,69,uniswap,0,1,Ind,NaN,"[{'aragonVotesPct': None, 'createdAt': '2023-09-02T01:39:35.503Z', 'deworkPoints': None, 'deworkTasksCompleted': None, 'discordScore': None, 'forumActivityScore': None, 'forumLikesReceived': None, 'forumPostCount': None, 'forumPostsReadCount': None, 'forumTopicCount': None, 'gitcoinHealthScore': None, 'karmaRank': 40, 'karmaScore': 69, 'offChainVotesPct': 94, 'onChainVotesPct': 68, 'percentile': 10.0, 'period': 'lifetime', 'proposalsDiscussed': None, 'proposalsInitiated': None, 'proposalsOnAragon': None, 'proposalsOnSnapshot': 0, 'updatedAt': '2024-03-02T02:26:07.763Z'}]",5,4627,NaN,0.00,2023-03-24T02:06:07.000Z,2023-03-24T02:06:07.000Z,[],"[{'name': 'forumActivityScore', 'pct': 0.0, 'score': 0}\n {'name': 'offChainVotesPct', 'pct': 44.927536231884055, 'score': 31}\n {'name': 'onChainVotesPct', 'pct': 55.072463768115945, 'score': 38}]",NaN,NaN,NaN,NaN,active,NaN,NaN,[],NaN,NaN,2022-11-23T00:12:09.614Z,False,NaN,[],NaN
14,0x054becf10c79afa4e60f75b187837be9fb83867d,3,75,47,5,1,NaN,arbitrum,coinruay.eth,NaN,NaN,False,False,86,arbitrumfoundation.eth,0,1,Ind,NaN,"[{'aragonVotesPct': None, 'createdAt': '2023-11-09T10:08:57.470Z', 'deworkPoints': None, 'deworkTasksCompleted': None, 'discordScore': Non